# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [1]:
!git clone https://github.com/timesler/facenet-pytorch

Cloning into 'facenet-pytorch'...
remote: Enumerating objects: 1267, done.
remote: Total 1267 (delta 0), reused 0 (delta 0), pack-reused 1267
Receiving objects: 100% (1267/1267), 22.88 MiB | 17.17 MiB/s, done.
Resolving deltas: 100% (620/620), done.


In [3]:
!pip install -q facenet_pytorch

     |████████████████████████████████| 1.9 MB 4.2 MB/s 


In [5]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os
%cd /content/facenet-pytorch

/content/facenet-pytorch


#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [10]:
data_dir = 'data/test_images'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

#### Determine if an nvidia GPU is available

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [32]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [ ]:
!wget https://www.elleman.vn/wp-content/uploads/2019/06/03/Angelina-Jolie-elle-man-featured-image.jpg -q -P /content/facenet-pytorch/data/test_images/angelina_jolie
!wget https://vcdn1-giaitri.vnecdn.net/2021/08/25/Angelina-Jolie-top-6702-1629859385.jpg?w=0&h=0&q=100&dpr=2&fit=crop&s=bPiM7bSi9ws-oOax4Y4Uew -q -P /content/facenet-pytorch/data/test_images/angelina_jolie
!wget https://storage0.dms.mpinteractiv.ro/media/401/341/5531/16234353/1/jolie-1.jpg?width=310 -q -P /content/facenet-pytorch/data/test_images/angelina_jolie
!wget https://vcdn1-giaitri.vnecdn.net/2021/08/25/Angelina-Jolie-top-6702-1629859385.jpg?w=0&h=0&q=100&dpr=2&fit=crop&s=bPiM7bSi9ws-oOax4Y4Uew -q -P /content/facenet-pytorch/data/test_images/angelina_jolie
!wget https://www.etonline.com/sites/default/files/styles/max_1280x720/public/images/2015-06/set_angelina_jolie_red_carpet_640.jpg?itok=SzToquIy -q -P /content/facenet-pytorch/data/test_images/angelina_jolie
!wget https://thumbs.dreamstime.com/b/angelina-jolie-angelina-jolie-26357396.jpg -q -P /content/facenet-pytorch/data/test_images/angelina_jolie


In [34]:
!wget "https://kenh14cdn.com/2020/6/4/angelina-jolie-1591262333644618329469.jpg" -q -P /content/facenet-pytorch/data/test_images/angelina_jolie
!wget "https://www.brides.com/thmb/efO7gj4mf1VDSWNPzX_VSc1ccKQ=/735x0/__opt__aboutcom__coeus__resources__content_migration__brides__public__brides-services__production__2016__10__24__580e6810382731317ade0896_blogs-aisle-say-angelina-jolie-weddings-main-64698e4cdeb64c07a8b53f4990e95173.jpg" -q -P /content/facenet-pytorch/data/test_images/angelina_jolie


In [27]:
!wget https://static-znews.zadn.vn/static/topic/person/bradley.jpg -q -P /content/facenet-pytorch/data/test_images/bradley_cooper
!wget https://media-cdn-v2.laodong.vn/Storage/NewsPortal/2022/6/18/1057851/Image.jpeg -q -P /content/facenet-pytorch/data/test_images/bradley_cooper
!wget https://phantom-marca.unidadeditorial.es/0aabd91083ebf7980b25a4579f7ac333/resize/1320/f/jpg/assets/multimedia/imagenes/2022/06/17/16554758566395.jpg -q -P /content/facenet-pytorch/data/test_images/bradley_cooper
!wget https://img.etimg.com/thumb/width-1200,height-900,imgsize-56168,resizemode-1,msid-91923282/news/international/us/bradley-cooper-plays-leonard-bernstein-in-the-biopic-maestro-and-is-unrecognisable.jpg -q -P /content/facenet-pytorch/data/test_images/bradley_cooper
!wget https://www.indiewire.com/wp-content/uploads/2022/06/Bradley-Cooper.jpg -q -P /content/facenet-pytorch/data/test_images/bradley_cooper


In [28]:
!wget https://cdn-aiogc.nitrocdn.com/rMYQrsZdIIazKavgLMeJFjBGGJYSrQyJ/assets/static/optimized/rev-66a7702/wp-content/uploads/2021/12/kate-siegel-header.jpg -q -P /content/facenet-pytorch/data/test_images/kate_siegel
!wget https://m.media-amazon.com/images/M/MV5BYTRhZGYzZDAtMDQ0MC00ZjVmLTkwN2ItYjk3NTRmNGJkMWY1XkEyXkFqcGdeQXVyNTQ1NTQyNjk@._V1_.jpg -q -P /content/facenet-pytorch/data/test_images/kate_siegel
!wget "https://i.pinimg.com/736x/27/4b/e6/274be6bf61d81a5a7b988cf23f379f0e.jpg" -q -P /content/facenet-pytorch/data/test_images/kate_siegel
!wget "https://i.pinimg.com/474x/43/b3/02/43b302d5022660454f54e42a2a14449d.jpg" -q -P /content/facenet-pytorch/data/test_images/kate_siegel


In [29]:
!wget "https://m.media-amazon.com/images/M/MV5BMTY4NTEwNDg1MV5BMl5BanBnXkFtZTgwODMwMDA0ODE@._V1_UY1200_CR165,0,630,1200_AL_.jpg" -q -P /content/facenet-pytorch/data/test_images/paul_rudd
!wget "https://www.themoviedb.org/t/p/w500/8eTtJ7XVXY0BnEeUaSiTAraTIXd.jpg" -q -P /content/facenet-pytorch/data/test_images/paul_rudd
!wget "https://deadline.com/wp-content/uploads/2016/04/paul-rudd.jpg" -q -P /content/facenet-pytorch/data/test_images/paul_rudd
!wget "https://www.nj.com/resizer/JcqW0gGlMGCtkpGUGvdYKMVr248=/1280x0/smart/cloudfront-us-east-1.images.arcpublishing.com/advancelocal/W5G7TNDQFNH45I6LNTDEHOM4R4.jpg" -q -P /content/facenet-pytorch/data/test_images/paul_rudd


In [30]:
!wget "https://www.themoviedb.org/t/p/w500/d3caK3l4UfbnzOxv95wLoFLZzMO.jpg" -q -P /content/facenet-pytorch/data/test_images/shea_whigham
!wget "https://www.purchase.edu/live/image/gid/34/width/640/3238_shea-whigham.rev.1484853205.jpg" -q -P /content/facenet-pytorch/data/test_images/shea_whigham
!wget "https://1.bp.blogspot.com/-vg6GlEDvbjA/TxeJzc8hTPI/AAAAAAAACxs/jKc9q6xWujA/s1600/Shea.jpg" -q -P /content/facenet-pytorch/data/test_images/shea_whigham
!wget "https://upload.wikimedia.org/wikipedia/commons/thumb/7/76/Shea_Whigham_in_2018.jpg/640px-Shea_Whigham_in_2018.jpg" -q -P /content/facenet-pytorch/data/test_images/shea_whigham


In [128]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=2,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Batch 1 of 10

/usr/local/lib/python3.7/dist-packages/facenet_pytorch/models/utils/detect_face.py:183: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  batch_boxes, batch_points = np.array(batch_boxes), np.array(batch_points)
/usr/local/lib/python3.7/dist-packages/facenet_pytorch/models/mtcnn.py:339: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  boxes = np.array(boxes)
/usr/local/lib/python3.7/dist-packages/facenet_pytorch/models/mtcnn.py:340: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or nd

Batch 7 of 10

/usr/local/lib/python3.7/dist-packages/facenet_pytorch/models/mtcnn.py:444: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  selected_boxes = np.array(selected_boxes)
/usr/local/lib/python3.7/dist-packages/facenet_pytorch/models/mtcnn.py:446: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  selected_points = np.array(selected_points)


Batch 10 of 10

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [129]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)


In [130]:
for params in resnet.parameters():
  params.requires_grad_(False)
  

In [131]:
for params in resnet.logits.parameters():
  params.requires_grad_(True)

#### Define optimizer, scheduler, dataset, and dataloader

In [138]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=32,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=32,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [139]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [140]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     1/1    | loss:    0.7333 | fps:    1.7296 | acc:    1.0000   

Epoch 1/8
----------
Train |     1/1    | loss:    1.0055 | fps:    6.6870 | acc:    0.6154   
Valid |     1/1    | loss:    0.6325 | fps:    3.7240 | acc:    1.0000   

Epoch 2/8
----------
Train |     1/1    | loss:    0.8153 | fps:    6.6509 | acc:    0.6923   
Valid |     1/1    | loss:    0.5416 | fps:    3.8258 | acc:    1.0000   

Epoch 3/8
----------
Train |     1/1    | loss:    0.6203 | fps:    6.9409 | acc:    0.7692   
Valid |     1/1    | loss:    0.4642 | fps:    3.7500 | acc:    1.0000   

Epoch 4/8
----------
Train |     1/1    | loss:    0.4899 | fps:    6.8301 | acc:    0.8462   
Valid |     1/1    | loss:    0.3971 | fps:    3.7575 | acc:    1.0000   

Epoch 5/8
----------
Train |     1/1    | loss:    0.3685 | fps:    6.6535 | acc:    0.9231   
Valid |     1/1    | loss:    0.3399 | fps:    3.6440 | acc:    1.0000   

Epoch 6/8
----------
Train |     1/1    | loss:    0.2

#test

In [117]:
!wget https://vcdn-ngoisao.vnecdn.net/2021/10/30/settopjolie-1635526336-6968-1635526893_1200x0.jpg -q -O test1.jpg 

In [118]:
from PIL import Image

In [141]:
test = mtcnn(Image.open("/content/facenet-pytorch/data/test_images/tranbaotri/6.jpg"), save_path='test1_mt.jpg').unsqueeze(0)
test = torch.concat([test, test], dim=0)

In [ ]:
resnet.eval()

In [143]:
torch.argmax(resnet(test))

tensor(3)

#Saving

In [149]:
!mkdir /content/facenet-pytorch/checkpoints

In [152]:
ckp_dir = "/content/facenet-pytorch/checkpoints"
torch.save(resnet.state_dict(), ckp_dir + 'model.pt')

#New person

In [127]:
!mkdir /content/facenet-pytorch/data/test_images/tranbaotri

#....


In [148]:
!git clone https://github.com/thangnch/MiAI_FaceRecog_2

Cloning into 'MiAI_FaceRecog_2'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 129 (delta 3), reused 9 (delta 0), pack-reused 114
Receiving objects: 100% (129/129), 2.70 MiB | 6.35 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from flask import Flask
from flask import render_template , request
from flask_cors import CORS, cross_origin
import torchvision.transforms as tf
import argparse
import os
import sys
import math
import pickle
import align.detect_face
from PIL import Image
import numpy as np
import cv2
import collections
from sklearn.svm import SVC
import base64

FACENET_MODEL_PATH = '/content/facenet-pytorch/checkpoints' + 'model.pt'
mtcnn = MTCNN(keep_all=True, post_process=False)
#get from config
num_classes = 4
class_name = ['jolie', 'john', 'badd', 'tri']
class_dict = dict([(class_name[i], num_classes[i]) for i in range(num_classes)])

def get_face(image):
  mtcnn = MTCNN(keep_all=True, post_process=False)
  face = mtcnn(image)[0]
  image = transforms.Resize((160, 160))((transforms.ToTensor()((Image.fromarray(face.permute(1, 2, 0).numpy().astype(np.uint8))))))
  image = Image.fromarray(face.permute(1, 2, 0).numpy().astype(np.uint8))
  return image



# Load the model
print('Loading feature extraction model')
facenet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=num_classes
).to(device)

facenet.load_state_dict(torch.load(FACENET_MODEL_PATH))
facenet.eval()

# Get input and output tensors by POST



app = Flask(__name__)
CORS(app)



@app.route('/')
@cross_origin()
def index():
    return "OK!";

@app.route('/recog', methods=['POST'])
@cross_origin()
def upload_img_file():
    if request.method == 'POST':
        name = "Unknown"
        og_image = request.form.get('image')
        image = get_face(og_image)
        image = tf.ToTensor()(image)
        logits = facenet(image)
        name = class_dict[torch.argmax(logits).item()]


        

        return name;


if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0',port='8000')



In [ ]:

get_face()
